In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import time

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

import sys
sys.path.append('..')
from helpers import secrets

In [18]:
youtube = build("youtube", "v3", developerKey=secrets.yt_key)

In [19]:
def youtubeSearch(query, token=None, maxResults=50, order="relevance", location=None, location_radius=None):

    search_response = youtube.search().list(
        q=query,
        type="video",
        pageToken=token,
        order = order,
        part="id,snippet",
        maxResults=maxResults,
        location=location,
        locationRadius=location_radius
    ).execute()
    
    return search_response

def storeResults(results):
    
    title = []
    channelId = []
    channelTitle = []
    categoryId = []
    viewCount = []
    likeCount = []
    dislikeCount = []
    commentCount = []
    category = []
    tags = []
    videos = []
    dates = []
    subs = []
    
    for search_result in results:
        if search_result["id"]["kind"] == "youtube#video":

            title.append(search_result['snippet']['title'])

            stats = youtube.videos().list(
                part='statistics, snippet',
                id=search_result['id']['videoId']).execute()
            
            stats_channel = youtube.channels().list(
                part='statistics',
                id=stats['items'][0]['snippet']['channelId']).execute()
                
            channelId.append(stats['items'][0]['snippet']['channelId']) 
            channelTitle.append(stats['items'][0]['snippet']['channelTitle']) 
            categoryId.append(stats['items'][0]['snippet']['categoryId']) 
            viewCount.append(stats['items'][0]['statistics']['viewCount'])
            dates.append(stats['items'][0]['snippet']['publishedAt'])
            subs.append(stats_channel['items'][0]['statistics'].get('subscriberCount', 0))
            
            try:
                likeCount.append(stats['items'][0]['statistics']['likeCount'])
            except:
                likeCount.append(0)
                
            try:
                dislikeCount.append(stats['items'][0]['statistics']['dislikeCount'])     
            except:
                dislikeCount.append(0)

            if 'commentCount' in stats['items'][0]['statistics'].keys():
                commentCount.append(stats['items'][0]['statistics']['commentCount'])
            else:
                commentCount.append(0)
         
            if 'tags' in stats['items'][0]['snippet'].keys():
                tags.append(stats['items'][0]['snippet']['tags'])
            else:
                tags.append(0)
                
    youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,
                    'categoryId':categoryId,'title':title,
                    'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,
                    'commentCount':commentCount, 'publishAt': dates, 'subs': subs}
 
    return youtube_dict

In [20]:
def writeCSV(results, filename, addHeader):
    import csv
    keys = sorted(results.keys())
    with open(filename, "a", newline="", encoding="utf-8") as output:
        writer = csv.writer(output, delimiter=",")
        if addHeader:
            writer.writerow(keys)
        writer.writerows(zip(*[results[key] for key in keys]))

In [23]:
q="mario bros"
items = []
pageToken = None
for i in range(10):
    response = youtubeSearch(q, pageToken)
    nextPage = response.get("nextPageToken", None)
    if nextPage is not None:
        pageToken = nextPage
        print(nextPage)
    time.sleep(5)
    items = items + response.get("items", [])
multiple_results = storeResults(items)


CKYEEAA
CNgEEAA


In [ ]:
print({
    'tags': multiple_results['tags'][0],
    'channelId': multiple_results['channelId'][0],
    'channelTitle': multiple_results['channelTitle'][0],
    'categoryId':multiple_results['categoryId'][0],
    'title':multiple_results['title'][0],
    'viewCount':multiple_results['viewCount'][0],
    'likeCount':multiple_results['likeCount'][0],
    'dislikeCount':multiple_results['dislikeCount'][0],
    'commentCount':multiple_results['commentCount'][0],
    'publishAt': multiple_results['publishAt'][0]
})

In [24]:
file = "results_data_videos.csv"
writeCSV(multiple_results, file, pageToken is None)